In [1]:
import requests
import pandas as pd
from urllib.parse import urlencode
import unidecode
import numpy as np


In [2]:
def query_batch_api(url: str, username: str, password: str, data: pd.DataFrame):

    # Create the request body as a dictionary from the DataFrame
    request_body = data.to_dict(orient="list")

    response = requests.post(url, json=request_body, auth=(username, password))
    
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 400:
        print(response.json()["detail"])
    else:
        print("Error occurred while querying the API.")
        return response

In [3]:
def get_index_nan_after_prepro(dataframe, text_feature):
    df = dataframe.copy()
    df[text_feature] = df[text_feature].map(unidecode.unidecode)

    # On passe tout en minuscule
    df[text_feature] = df[text_feature].str.lower()

    # define replacement patterns
    replacements = {
        # Libellé vide de sens fournit par Christine (DOIT ETRE FAIT EN AMONT DU MODELE EN JAVA)
        r"\bidem\b|\bvoir ci dessous\b|\[vide\]|\bundefined\b|\bpas d objet\b|\(voir ci dessus\)|\(voir extrait siege social\\/etablissement principal\)|\bcf activite principale\b|\bcf activite principale et objet\b|\bcf activites de l entreprise\b|\bcf activites principales de l entreprise\b|\bcf actvites principales\b|\bcf k bis\b|\bcf le principales activites de l  entreprise\b|\bcf le sprincipale activites de l  entreprise\b|\bcf le sprincipales activites de l  entreprise\b|\bcf les activites principales de l  entreprise\b|\bcf les ppales activites de l  entreprise\b|\bcf les ppales activites de la ste\b|\bcf les principale activites de l  entreprise\b|\bcf les principales activites\b|\bcf les principales activites de l  entreprise\b|\bcf les principales activites de l  entreprises\b|\bcf les principales activites ppales de l  entreprise\b|\bcf les principales activtes de l  entreprise\b|\bcf les principales acttivites de l  entreprise\b|\bcf les prinipales activites de l  entreprise\b|\bcf lesprincipales activites de l  entreprise\b|\bcf objet\b|\bcf obs\b|\bcf principales activite de l  entreprise\b|\bcf principales activites de l  entreprise\b|cf rubrique \"principales activites de l entreprise\" idem|cf rubrique n2 ci dessus \(743b\)|\bcf supra\b|\bcf ci  dessus\b|\bcommerce de detail, idem case 2\b|\bextension a: voir ci dessus\b|\bid\b|\bid principales activites\b|\bid principales activites de l  entreprise\b|\bidem ci dessus\b|idem \( voir principales activites\)|\bidem  dessus\b|\bidem 1ere page\b|\bidem a principales activites de l  entreprise\b|\bidem activiet eprincipale\b|\bidem activite\b|\bidem activite 1ere page\b|\bidem activite ci  dessus\b|\bidem activite de l  entreprise\b|\bidem activite enoncee ci  dessus\b|\bidem activite entreprise\b|\bidem activite generales\b|\bidem activite premiere page\b|\bidem activite principale\b|\bidem activite princippale\b|\bidem activite prinicpale\b|\bidem activite sur 1ere page\b|\bidem activites ci dessus\b|\bidem activites declarees au siege et principal\b|\bidem activites enoncees ci dessus\b|\bidem activites entreprise\b|\bidem activites principales\b|\bidem activites principales de l entreprise\b|\bidem activites siege\b|\bidem activte principale\b|\bidem activtie 1ere page\b|\bidem au siege\b|\bidem au siege social\b|\bidem aux principales actiivtes\b|\bidem aux principales activites\b|\bidem case 13\b|\bidem ci dessous\b|\bidem ci dessus enoncee\b|\bidem cidessus\b|\bidem objet\b|\bidem premiere page\b|\bidem pricincipales activites de l entreprise\b|\bidem pricipales activites\b|\bidem principale activite\b|\bidem principales activite de l entreprise\b|\bidem principales activite de l entreprises\b|\bidem principales activite l entreprise\b|\bidem principales activites\b|\bidem principales activites citees ci dessus\b|\bidem principales activites de l entreprises\b|idem principales activites de l entreprise\(objet\)|\bidem principales activites et objet social\b|\bidem principales activitse de l entreprise\b|\bidem que celle decrite plus haut\b|\bidem que ci dessus\b|\bidem que l activite decrite plus haut\b|\bidem que les activites principales\b|\bidem que les activites principales ci dessus\b|\bidem que les activitges principales\b|\bidem que les principales activites\b|\bidem que les principales activites de l entreprise\b|\bidem que pour le siege\b|\bidem rubrique principales activites de l entreprise\b|\bidem siege\b|idem siege \+ voir observation|\bidem siege et ets principal\b|\bidem siege social\b|idem siege, \(\+ articles americains\)|\bidem societe\b|\bidem voir activite principale\b|\bidem voir ci dessus\b|\bidentique a l objet social indique en case 2 de l imprime m2\b|\bidm ci dessus\b|\bnon indiquee\b|\bnon precise\b|\bnon precisee\b|\bnon precisees\b|\bvoir 1ere page\b|\bvoir activite ci dessus\b|\bvoir activite principale\b|\bvoir activite principale ci dessus\b|\bvoir activites principales\b|\bvoir cidessus\b|\bvoir idem ci dessus\b|\bvoir objet social\b|\bvoir page 1\b|\bvoir page precedente\b|\bvoir plus haut\b|\bvoir princiale activite\b|\bvoir princiales activites\b|\bvoir princiapales activites\b|\bvoir princiaples activites\b|\bvoir principale activite\b|\bvoir principales activites\b|\bvoir principales activites de l entreprise\b|\bvoir principales actvites\b|\bvoir principalesactivites\b|\bvoir principles activites\b|\bvoir rubrique principales activites de l entreprise\b|\bvoir sur la 1ere page\b|\bvoir dessus\b|voir: \"principales activite de l entreprise\"|voir: \"principales activites de l entreprises\"|voir: \"principales activites de l entrprise\"|voir: \"principales activites en entreprise\"|\bconforme au kbis\b|\bsans changement\b|\bsans activite\b|\bsans acitivite\b|\bactivite inchangee\b|\bactivites inchangees\b|\bsiege social\b|\ba definir\b|\ba preciser\b|\bci dessus\b|\bci desus\b|\bci desssus\b|\bvoir activit principale\b|\bidem extrait kbis\b|\bn a plus a etre mentionne sur l extrait decret\b|\bcf statuts\b|\bactivite principale case\b|\bactivites principales case\b|\bactivite principale\b|\bactivites principales\b|\bvoir case\b|\baucun changement\b|\bsans modification\b|\bactivite non modifiee\b|\bactivite identique\b|\bpas de changement\b|\bcode\b|\bape\b|\bnaf\b|\binchangee\b|\binchnagee\b|\bkbis\b|\bk bis\b|\binchangees\b|\bnp\b|\binchange\b|\bnc\b|\bxx\b|\bxxx\b|\binconnue\b|\binconnu\b|\bvoir\b|\bannexe\b|\bmo\b|\biem\b|\binchanges\b|\bactivite demeure\b|\bactivite inchangée\b|\bcase precedente\b|\bidem cadre precedent\b|\bactivite demeure\b|\bactivite inchangée\b|\bnon renseignee\b|\bneant\b|\bnon renseigne\b": " ",
        # supprime hyphen pour les mots comme e-commerce
        r"e-": "e",
        # accole le e pour les mots comme e-commerce
        r"\be\s": " e",
        # On supprime toutes les ponctuations
        r"[^\w\s]": " ",
        # On supprime certains mots sans sens (DOIT ETRE FAIT DANS LE PREPROCESSING EN JAVA)
        r"\bcode\b|\bcadre\b|\bape\b|\bape[a-z]{1}\b|\bnaf\b|\binchangee\b|\binchnagee\b|\bkbis\b|\bk bis\b|\binchangees\b|\bnp\b|\binchange\b|\bnc\b|\bidem\b|\bxx\b|\bxxx\b|\baa\b|\baaa\b|\bidem cadre precedent\b|\bidem case\b|\binchanges\b|\bmo\b|\biem\b|\bci dessus\b|\bet\b": "",
        # On supprime tous les chiffres
        r"[\d+]": " ",
    }

    # apply replacements to text_feature column
    for pattern, replacement in replacements.items():
        df[text_feature] = df[text_feature].str.replace(
            pattern, replacement, regex=True
        )

    # On supprime les mots d'une seule lettre
    df[text_feature] = df[text_feature].apply(
        lambda x: " ".join([w for w in x.split() if len(w) > 1])
    )

    # define replacement patterns
    replacements = {
        # On supprime les espaces multiples
        r"\s\s+": " ",
        # On strip les libellés
        r"^\s+|\s+$": "",
        # On remplace les empty string par des NaN
        r"^\s*$": np.nan,
    }

    # apply replacements to text_feature column
    for pattern, replacement in replacements.items():
        df[text_feature] = df[text_feature].replace(
            pattern, replacement, regex=True
        )

    return df[pd.isnull(df).any(axis=1)].index

In [4]:
data = pd.read_csv("../data/Jeu_codif_APE_S4_20230606.csv")
data['activ_nat_et'] = data['activ_nat_et'].apply(lambda x: "{:02d}".format(int(x)) if pd.notnull(x) else np.nan)
data = data[["activ_pr_lib_et", "liasse_type", "activ_nat_et", "activ_surf_et", "evenement_type", "apet_finale"]]
data.columns = ["text_description" , "type_", "nature","surface", "event","code"]
data = data.dropna(subset=["text_description"])
data['nature'] = data['nature'].apply(lambda x: "{:02d}".format(int(x)) if pd.notnull(x) else "")
data['surface'] = data['surface'].apply(lambda x: "" if pd.isnull(x) else ("1" if x < 121 else ("2" if 121 <= x <= 400 else ("3" if 401 <= x <= 2500 else "4"))))
idx = get_index_nan_after_prepro(data, "text_description")
data = data.drop(idx)
data.to_csv("Jeu_codif_APE_S4_20230606_processed.csv")


In [9]:
data = pd.read_csv("Jeu_codif_APE_S4_20230606_processed.csv", index_col=[0])
data['nature'] = data['nature'].apply(lambda x: "{:02d}".format(int(x)) if pd.notnull(x) else "")
data['surface'] = data['surface'].apply(lambda x: "" if pd.isnull(x) else ("1" if x < 121 else ("2" if 121 <= x <= 400 else ("3" if 401 <= x <= 2500 else "4"))))


In [10]:
data

,text_description,type_,nature,surface,event,code
0,Nettoyage tout type de locaux,X,,,01P,8121Z
1,SOUDURE,M,99,,01P,3320A
2,"Retouche et confection de rideaux, stores bate...",M,04,,01P,1392Z
3,"prestations de services, déménagements",M,99,,01P,4942Z
4,Livraison uber eats,C,99,,01P,5320Z
...,...,...,...,...,...,...
440422,"L'administration et l'exploitation par bail, l...",G,,,01M,6820B
440423,Centre d'appel.,C,99,,01M,8220Z
440424,"l'acquisition, l'administration et la gestion,...",G,,,01M,6820B
440425,Production d'électricité photovoltaïque inféri...,I,,,01P,3511Z


In [11]:
url = "https://codification-ape.lab.sspcloud.fr/evaluation"
query_batch_api(url, "codification-ape","codification-sirene4", data)
#query_batch_api(url, "toto","toto",data)

Error occurred while querying the API.


<Response [502]>

In [35]:
df = pd.read_csv("../data/Jeu_codif_APE_S4_20230606_processed.csv",index_col=[0])
df

,text_description,type_,nature,surface,event,code
0,Nettoyage tout type de locaux,X,NaN,NaN,01P,8121Z
1,SOUDURE,M,99.0,NaN,01P,3320A
2,"Retouche et confection de rideaux, stores bate...",M,4.0,NaN,01P,1392Z
3,"prestations de services, déménagements",M,99.0,NaN,01P,4942Z
4,Livraison uber eats,C,99.0,NaN,01P,5320Z
...,...,...,...,...,...,...
440422,"L'administration et l'exploitation par bail, l...",G,NaN,NaN,01M,6820B
440423,Centre d'appel.,C,99.0,NaN,01M,8220Z
440424,"l'acquisition, l'administration et la gestion,...",G,NaN,NaN,01M,6820B
440425,Production d'électricité photovoltaïque inféri...,I,NaN,NaN,01P,3511Z
